# We need some libraries and initial parameters...

In [4]:
library(SparkR)
library(leaflet)
library(sp)
options(digits=15)


the_lat = 39.976057
the_lng = 116.330243
the_zoom = 15
epsilon = 200
source("pbfe.R")


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, endsWith, intersect, rank, rbind, sample,
    startsWith, subset, summary, table, transform



# Let load some data...

In [5]:
data = read.csv("sample_small.csv")
names(data) = c("ID","lat","lng")
head(data)

ID,lat,lng
1,39.5905830,116.2719450
2,39.6334840,116.7282400
3,39.6558133,116.6641916
4,39.6596780,116.6711490
5,39.6654583,116.3098399
6,39.6728416,116.3086649


# and visualize them in a map...

In [6]:
map = leaflet() %>%
  addTiles() %>%
  addCircleMarkers(lng=data$lng, lat=data$lat,weight=2,fillOpacity=1,color="blue",radius=2)

file = 'map1.html'
htmlwidgets::saveWidget(map, file = file, selfcontained = F)
IRdisplay::display_html(paste("<iframe width=100% height=400 src=' ", file, " ' ","/>"))

https://ec2-35-162-64-98.us-west-2.compute.amazonaws.com:8888/home/ubuntu/notebooks/files/map1.html

# Connecting with Simba and getting a SQLContext...

In [7]:
sc <- sparkR.init("local[*]", "SparkR")
sqlContext <- sparkRSQL.init(sc)

Launching java with spark-submit command /opt/Simba/bin/spark-submit   sparkr-shell /tmp/RtmpzeJA5w/backend_port58fb4c057ff4 


# Let read the data into Simba and apply a transformation...

In [8]:
dataRDD = SparkR:::textFile(sc,"sample_small.csv")
dataRDD = SparkR:::map(dataRDD, transformCoords)

# Now We create a DataFrame and cache the data...

In [9]:
schema <- structType(structField("id", "double"), structField("lng", "double"), structField("lat", "double"))
points <- createDataFrame(sqlContext, dataRDD, schema = schema)
cache(points)

DataFrame[id:double, lng:double, lat:double]

# Let's have a look...

In [10]:
head(points)
count(points)

id,lng,lat
0,-300629.930290752,4419848.70124370
1,-336418.933794024,4429641.88724925
2,-296625.402561355,4430225.49965138
3,-301876.547452500,4433286.74767265
4,-301232.904159629,4433654.10554999
5,-332247.212576558,4437621.06989079


[1] 899

# Now, We need register a pair of temporal tables...

In [11]:
registerTempTable(points, "p1")
registerTempTable(points, "p2")

# It is time to execute the SQL statement...

```SQL
SELECT
    * 
FROM 
    p1 
DISTANCE JOIN 
    p2
ON 
    POINT(p2.lng, p2.lat) IN CIRCLERANGE(POINT(p1.lng, p1.lat), epsilon)
WHERE 
    p2.id < p1.id
```

In [12]:
sql = paste0("SELECT * FROM p1 DISTANCE JOIN p2 ON POINT(p2.lng, p2.lat) IN CIRCLERANGE(POINT(p1.lng, p1.lat), ",epsilon,") WHERE p2.id < p1.id")
pairs = sql(sqlContext,sql)
head(pairs)
nrow(pairs)

id,lng,lat,id,lng,lat
26,-332354.697252151,4456150.25607556,24,-332269.409834687,4456108.23475632
45,-334252.427014354,4459698.61791281,44,-334344.731428849,4459655.45188833
46,-334242.745346436,4459715.58979402,44,-334344.731428849,4459655.45188833
46,-334242.745346436,4459715.58979402,45,-334252.427014354,4459698.61791281
76,-332954.898512807,4460728.70522263,74,-333132.922385918,4460639.54370124
121,-353770.269435442,4466969.59768473,120,-353766.652493862,4466963.17659202


[1] 923

# Now We need to calculate the disk locations for each pair...

In [13]:
centers <- SparkR:::map(pairs, calculateDisk)
schema <- structType(structField("id1", "double"), structField("id2", "double"), structField("lng1", "double"), structField("lat1", "double"), structField("lng2", "double"), structField("lat2", "double"))
d <- createDataFrame(sqlContext, centers, schema = schema)
head(d)
count(d)

id1,id2,lng1,lat1,lng2,lat2
26,24,-332273.170148452,4456208.16403151,-332350.936938386,4456050.32680037
45,44,-334262.128211716,4459599.08959136,-334335.030231487,4459754.98020978
46,44,-334252.801197014,4459616.09667934,-334334.675578271,4459754.94500301
46,45,-334161.140917582,4459657.79085639,-334334.031443208,4459756.41685044
76,74,-333039.675560248,4460675.66889331,-333048.145338477,4460692.58003055
121,120,-353681.392035067,4467015.43225359,-353855.529894237,4466917.34202316


[1] 923

# Let's collect the data back to LatLong coordinates...

In [14]:
centers_lnglat <- SparkR:::map(centers, transformCenters)
disks <- as.data.frame(createDataFrame(sqlContext,centers_lnglat))
names(disks) = c("id1","id2","lng1","lat1","lng2","lat2")
head(disks)
nrow(disks)

id1,id2,lng1,lat1,lng2,lat2
26,24,116.286172307920,39.8304366534153,116.285476216861,39.8289598550172
45,44,116.258979560390,39.8585969529074,116.257942352089,39.8599094498003
46,44,116.259065430375,39.8587570188828,116.257946481424,39.8599094837162
46,45,116.260068282656,39.8592166161994,116.257952034760,39.8599231771293
76,74,116.271696610161,39.8693462236874,116.271577630914,39.8694880711692
121,120,116.025807571434,39.9052021013708,116.023929528632,39.9041577846355


[1] 923

# Let's have a look at the results...

In [15]:
p = sort(unique(c(disks$id1,disks$id2)))
data2 = data[p,]
map = leaflet() %>% setView(lat = the_lat, lng = the_lng, zoom = the_zoom) %>% addTiles() %>% 
        addCircles(lng=disks$lng1, lat=disks$lat1, weight=2, fillOpacity=0.25, color="red", radius = epsilon/2) %>%
        addCircles(lng=disks$lng2, lat=disks$lat2, weight=2, fillOpacity=0.25, color="red", radius = epsilon/2) %>%
        addCircleMarkers(lng=data$lng, lat=data$lat, weight=2, fillOpacity=1,radius = 2) %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius = 2, popup=paste(data2$ID)) %>% 
        addProviderTiles("Esri.WorldImagery", group = "ESRI") %>% 
        addLayersControl(baseGroup = c("OSM(default)", "ESRI"))

file = 'map2.html'
htmlwidgets::saveWidget(map, file = file, selfcontained = F)
IRdisplay::display_html(paste("<iframe width=100% height=400 src=' ", file, " ' ","/>"))

https://ec2-35-162-64-98.us-west-2.compute.amazonaws.com:8888/home/ubuntu/notebooks/files/map2.html

In [16]:
registerTempTable(d, "d")
registerTempTable(points, "p")

In [17]:
sql = paste0("SELECT d.lng1 AS lng, d.lat1 AS lat, id AS id_member FROM d DISTANCE JOIN p ON POINT(p.lng, p.lat) IN CIRCLERANGE(POINT(d.lng1, d.lat1), ",(epsilon/2)+0.01,")")
mdisks = sql(sqlContext,sql)
registerTempTable(mdisks, "m")
sql = "SELECT lng, lat FROM m GROUP BY lng, lat HAVING count(id_member) >= 3"
mdisks1 = sql(sqlContext,sql)

In [18]:
sql = paste0("SELECT d.lng2 AS lng, d.lat2 AS lat, id AS id_member FROM d DISTANCE JOIN p ON POINT(p.lng, p.lat) IN CIRCLERANGE(POINT(d.lng2, d.lat2), ",(epsilon/2)+0.01,")")
mdisks = sql(sqlContext,sql)
registerTempTable(mdisks, "m")
sql = "SELECT lng, lat FROM m GROUP BY lng, lat HAVING count(id_member) >= 3"
mdisks2 = sql(sqlContext,sql)

In [19]:
mdisks = as.data.frame(rbind(mdisks1, mdisks2))
id = seq(1,nrow(mdisks))
mdisks$id = id

coordinates(mdisks) = ~lng+lat
proj4string(mdisks) = mercator
mdisks = spTransform(mdisks, wgs84)
mdisks$lng1 = coordinates(mdisks)[,1]
mdisks$lat1 = coordinates(mdisks)[,2]

In [20]:
map = leaflet() %>% setView(lat = the_lat, lng = the_lng, zoom = the_zoom) %>% addTiles() %>% 
        addCircles(lng=mdisks$lng1, lat=mdisks$lat1, weight=2, fillOpacity=0.25, color="blue", radius = epsilon/2) %>%
        addCircleMarkers(lng=data$lng, lat=data$lat, weight=2, fillOpacity=1,radius = 2) %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius = 2) %>% 
        addProviderTiles("Esri.WorldImagery", group = "ESRI") %>% 
        addLayersControl(baseGroup = c("OSM(default)", "ESRI"))

file = 'map3.html'
htmlwidgets::saveWidget(map, file = file, selfcontained = F)
IRdisplay::display_html(paste("<iframe width=100% height=400 src=' ", file, " ' ","/>"))

In [21]:
v1 = "656,659,660"

v1 = as.numeric(unlist(strsplit(v1,',')))

v1

v2 = c(656,657,659,660)

v2

prod(is.element(v1, v2))

4:3

[1] 656 659 660

[1] 656 657 659 660

[1] 1

[1] 4 3

In [22]:
m <- as.data.frame(rbind(mdisks1, mdisks2))
m$id = seq(1,nrow(m))
m = createDataFrame(sqlContext, m)
head(m)
count(m)
registerTempTable(m, "m")

head(points)
count(points)

sql = paste0("SELECT m.id AS mid, p.id AS pid FROM m DISTANCE JOIN p ON POINT(p.lng, p.lat) IN CIRCLERANGE (POINT(m.lng, m.lat), ",(epsilon/2)+0.01,") ORDER BY mid, pid")
t = sql(sqlContext,sql)
head(t, 30)
count(t)

lng,lat,id
-326817.249165109,4473000.32009028,1
-326733.479114837,4472020.94361649,2
-326441.677006837,4473064.94544083,3
-326769.903381107,4472499.05395445,4
-326466.460629624,4473654.18613708,5
-326636.726938150,4474740.14306566,6


[1] 1248

id,lng,lat
0,-300629.930290752,4419848.70124370
1,-336418.933794024,4429641.88724925
2,-296625.402561355,4430225.49965138
3,-301876.547452500,4433286.74767265
4,-301232.904159629,4433654.10554999
5,-332247.212576558,4437621.06989079


[1] 899

mid,pid
1,569
1,584
1,591
2,447
2,451
2,457
2,458
2,464
2,467
2,470


[1] 5573

In [23]:
library(sqldf)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite
Loading required package: DBI


In [24]:
t = as.data.frame(t)
g = sqldf("SELECT mid, group_concat(CAST(pid AS INT)) AS pids FROM t GROUP BY mid ORDER BY count(pid)")
head(g)
nrow(g)
tail(g)

Loading required package: tcltk


mid,pids
1,"569,584,591"
3,"587,590,610"
9,"864,865,867"
10,"550,551,556"
16,"618,627,632"
17,"698,699,708"


[1] 1248

,mid,pids
1243,594,"403,405,406,407,409,411,412,413,415,417"
1244,722,"403,405,406,407,409,411,412,413,415,417"
1245,730,"403,405,406,407,409,411,412,413,415,417"
1246,907,"403,405,406,407,409,411,412,413,415,417"
1247,1024,"438,443,444,447,451,457,458,464,467,470"
1248,1214,"438,443,444,447,451,457,458,464,467,470"


In [25]:
n = c()
r = nrow(g)
for(i in 1:r){
    disk_i = as.numeric(unlist(strsplit(g[i,2], ',')))
    flag = 1
    for(j in (i+1):r){
        disk_j = as.numeric(unlist(strsplit(g[j,2], ',')))
        # print(paste0("Disk 1: ", disk_i, " Disk 2: ", disk_j, " Result: ", is.element(disk_i, disk_j)))
              
        if(prod(is.element(disk_i, disk_j)) == 1){
            flag = 0
            break
        }
    }
    if(flag == 1){
        n = c(n, i)
    }
}
n = c(n, r)
n = unique(n)

In [26]:
g = g[n,]

m = as.data.frame(m)
maximal = sqldf("SELECT lng, lat, pids FROM g JOIN m ON(id = mid)")
head(maximal)
nrow(maximal)

coordinates(maximal) = ~lng+lat
proj4string(maximal) = mercator
maximal = spTransform(maximal, wgs84)
maximal$lng1 = coordinates(maximal)[,1]
maximal$lat1 = coordinates(maximal)[,2]

lng,lat,pids
-326512.392249577,4471574.61187925,"394,403,405"
-326924.099686826,4473135.22402740,"584,591,619"
-316480.164435932,4472109.23487005,"614,626,633"
-325995.046184977,4472418.12312925,"523,535,537"
-327206.790234772,4475419.52784507,"753,754,759"
-326924.932682156,4471864.92765717,"416,418,432"


[1] 161

In [27]:
map = leaflet() %>% setView(lat = the_lat, lng = the_lng, zoom = the_zoom) %>% addTiles() %>% 
        addCircles(lng=maximal$lng1, lat=maximal$lat1, weight=2, fillOpacity=0.25, color="orange", radius = epsilon/2, popup = maximal$pids) %>%
        addCircleMarkers(lng=data$lng, lat=data$lat, weight=2, fillOpacity=1,radius = 2) %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius = 2) %>% 
        addProviderTiles("Esri.WorldImagery", group = "ESRI") %>% 
        addLayersControl(baseGroup = c("OSM(default)", "ESRI"))

file = 'map4.html'
htmlwidgets::saveWidget(map, file = file, selfcontained = F)
IRdisplay::display_html(paste("<iframe width=100% height=400 src=' ", file, " ' ","/>"))

In [29]:
map = leaflet() %>% setView(lat = the_lat, lng = the_lng, zoom = the_zoom) %>% addTiles() %>% 
        addCircleMarkers(lng=data$lng, lat=data$lat, weight=2, fillOpacity=1,radius = 2, group = "Points") %>%
        
        addCircles(lng=disks$lng1, lat=disks$lat1, weight=2, fillOpacity=0.10, color="red", radius = epsilon/2, group = "Initial set") %>%
        addCircles(lng=disks$lng2, lat=disks$lat2, weight=2, fillOpacity=0.10, color="red", radius = epsilon/2, group = "Initial set") %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius=2, group = "Initial set") %>% 
        
        addCircles(lng=mdisks$lng1, lat=mdisks$lat1, weight=2, fillOpacity=0.20, color="blue", radius = epsilon/2, group = "Prune less than mu") %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius=2, group = "Prune less than mu") %>% 
        
        addCircles(lng=maximal$lng1, lat=maximal$lat1, weight=2, fillOpacity=0.40, color="orange", radius = epsilon/2, popup = maximal$pids, group = "Prune redundant") %>%
        addCircleMarkers(lng=data2$lng, lat=data2$lat, weight=2, fillOpacity=1, color="purple", radius=2, group = "Prune redundant") %>% 
        
        addProviderTiles("Esri.WorldImagery", group = "ESRI") %>% 
        addLayersControl(baseGroup = c("OSM(default)", "ESRI"),
                        overlayGroups = c("Initial set", "Prune less than mu", "Prune redundant", "Points"),
                        options = layersControlOptions(collapsed = FALSE, autoZIndex = FALSE)) 


file = 'map5.html'
htmlwidgets::saveWidget(map, file = file, selfcontained = F)
IRdisplay::display_html(paste("<iframe width=100% height=400 src=' ", file, " ' ","/>"))